### 숙제

OpenWeather API 회원가입 후 받은 API_KEY로 서울시 날씨에 대한 다른 정보도 출력해주세요!
(https://openweathermap.org/api) 

In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import json
load_dotenv()

# code here 

In [1]:
import requests


class WeatherApiClient:
    """
    OpenWeatherMap API 클라이언트를 사용하여 날씨 데이터를 가져오는 클래스입니다.
    """

    def __init__(self, api_key: str):
        self.base_url = "http://api.openweathermap.org/data/2.5"
        if api_key is None:
            raise Exception("API 키는 None으로 설정할 수 없습니다.")
        self.api_key = api_key

    def get_city(self, city_name: str, temperature_units: str = "metric") -> dict:
        """
        지정된 도시의 최신 날씨 데이터를 가져옵니다.

        Parameters:
        - city_name (str): 날씨 정보를 조회할 도시 이름.
        - temperature_units (str): 온도 단위 (기본값은 'metric'으로 섭씨 기준).
                                   'metric'은 섭씨, 'imperial'은 화씨, 'standard'는 켈빈 단위를 의미합니다.

        Returns:
        - dict: 요청한 도시의 날씨 데이터가 포함된 JSON 응답을 반환합니다.

        Raises:
        - Exception: API 요청이 실패한 경우 상태 코드와 응답 메시지와 함께 예외가 발생합니다.
        """
        params = {"q": city_name, "units": temperature_units, "appid": self.api_key}
        response = requests.get(f"{self.base_url}/weather", params=params)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(
                f"Open Weather API에서 데이터를 추출하지 못했습니다. 상태 코드: {response.status_code}. 응답: {response.text}"
            )

In [13]:
import pandas as pd 
from dotenv import load_dotenv
import os 
load_dotenv()

weather_api_client = WeatherApiClient(api_key = os.environ.get("API_KEY")) 
data = weather_api_client.get_city("Sydney")
pd.json_normalize(data)

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 802, 'main': 'Clouds', 'description': ...",stations,10000,1731905625,39600,2147714,Sydney,200,151.2073,-33.8679,...,1005,4.43,97,5.49,41,2,2003572,AU,1731868917,1731919138


## 한도시 (Sydney)

In [14]:
import pandas as pd 
from dotenv import load_dotenv
import os 
load_dotenv()

weather_api_client = WeatherApiClient(api_key = os.environ.get("API_KEY")) 
data = weather_api_client.get_city("Sydney") 

# 간단한 전처리
df_weather = pd.json_normalize(data) # JSON 데이터를 Pandas 형태로 변환 
df_weather["measured_at"] = pd.to_datetime(df_weather["dt"], unit="s") + pd.Timedelta(hours=9)  # 기준시간 
df_weather["dt"] = df_weather["measured_at"].dt.strftime('%Y%m%d') # 기준년월일 (YYYYMMDD) 
df_weather["time"] = df_weather["measured_at"].dt.strftime('%H%M%S') # 기준년월일 (HHHHMMSS) 
df_selected = df_weather[["dt", "time", "measured_at", "id", "name", "main.temp", "main.humidity", "wind.speed"]]
df_selected = df_selected.rename( # 컬럼명 수정 
    columns={"name": "city", "main.temp": "temperature", "main.humidity": "humidity", "wind.speed": "wind_speed"}
)

# DataFrame 생성 및 CSV 파일로 저장
df_selected.to_csv('weather_api.csv', index=False)
df_selected.head()


,dt,time,measured_at,id,city,temperature,humidity,wind_speed
0,20241118,135345,2024-11-18 13:53:45,2147714,Sydney,29.39,43,4.43


## 여러 도시

In [17]:
cities = ["Sydney", "London", "New York", "Tokyo", "Toronto"]

all_data = []

# Collect data for each city and store it in a list
for city in cities:
    data = weather_api_client.get_city(city)
    all_data.append(data)

# Normalize all data at once
df_weather = pd.json_normalize(all_data)
df_weather["measured_at"] = pd.to_datetime(df_weather["dt"], unit="s") + pd.Timedelta(hours=9)  # 기준시간 
df_weather["dt"] = df_weather["measured_at"].dt.strftime('%Y%m%d')
df_weather["time"] = df_weather["measured_at"].dt.strftime('%H%M%S')
df_selected = df_weather[["dt", "time", "measured_at", "id", "name", "main.temp", "main.humidity", "wind.speed"]]
df_selected = df_selected.rename(
    columns={"name": "city", "main.temp": "temperature", "main.humidity": "humidity", "wind.speed": "wind_speed"}
)

# Save the DataFrame to a CSV file
df_selected.to_csv('weather_api.csv', index=False)
df_selected.head()


,dt,time,measured_at,id,city,temperature,humidity,wind_speed
0,20241118,140415,2024-11-18 14:04:15,2147714,Sydney,28.47,47,4.43
1,20241118,135709,2024-11-18 13:57:09,2643743,London,7.24,91,0.00
2,20241118,140325,2024-11-18 14:03:25,5128581,New York,10.03,53,2.06
3,20241118,135653,2024-11-18 13:56:53,1850144,Tokyo,13.76,56,6.69
4,20241118,140544,2024-11-18 14:05:44,6167865,Toronto,9.76,89,6.69
